In [1]:
import requests
from shapely.geometry import LineString
import pandas as pd
import json
import osmnx as ox
import pyrosm
import folium
import numpy as np
import networkx as nx

ox.__version__

useful_tags_path = ['bridge', 'tunnel', 'oneway', 'lanes', 'ref', 'name',
                    'highway', 'maxspeed', 'service', 'access', 'area',
                    'landuse', 'width', 'est_width', 'junction', 'traffic_signals']
ox.utils.config(useful_tags_path)

In [2]:
def get_full_traffic_data(G):
    
    nodes, edges = ox.graph_to_gdfs(G)
    
    edges.name = (edges.name).astype(str)
    
    names = edges.name.unique()
    
    for name in names:
        get_specific_traffic_data(edges, name)
        
    new_attrs = ['current_speed_kmph', 'free_flow_speed_kmph', 'traffic_confidence', 'road_closure']
    for attr in new_attrs:
        nx.set_edge_attributes(G, edges[attr], attr)
        

def get_specific_traffic_data(edges, name):
    # Create request URL
    BASE_URL = "https://api.tomtom.com/"
    API_KEY = "ThfURaOhGRaty7mx9AKALwrbyIZApHd8"
    #zoom = 22
    TRAFFIC_URL = "traffic/services/4/flowSegmentData/absolute/22/json?key="
    UNIT = "unit=kmph"
    ROAD_CLOSURE = "roadClosure=True"
    PARAMS = "&" + UNIT + "&" + ROAD_CLOSURE
    
    coordinates = list(edges.loc[edges['name'] == name].iloc[0].geometry.coords)[0]
    longitude = str(coordinates[0])
    latitude = str(coordinates[1])
    location = "point=" + latitude + ","+ longitude

    request_url = BASE_URL + TRAFFIC_URL + API_KEY + "&" + location + PARAMS
    # Get data
    response = requests.get(request_url)
    data = response.json()['flowSegmentData']
    traffic = pd.json_normalize(data)
    
    edges.loc[edges['name'] == name, 'road_type'] = traffic.frc[0]
    edges.loc[edges['name'] == name, 'current_speed_kmph'] = traffic.currentSpeed[0]
    edges.loc[edges['name'] == name, 'free_flow_speed_kmph'] = traffic.freeFlowSpeed[0]
    edges.loc[edges['name'] == name, 'traffic_confidence'] = traffic.confidence[0]
    edges.loc[edges['name'] == name, 'road_closure'] = traffic.roadClosure[0]

In [3]:
ADDRESS = '30 Aldwych, London WC2B 4BG'
G = ox.graph_from_address(ADDRESS , network_type="drive", dist=2000)

/Users/khalid-alsheeb/opt/anaconda3/envs/kurf/lib/python3.10/site-packages/osmnx/utils_geo.py:280: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in geometry:
/Users/khalid-alsheeb/opt/anaconda3/envs/kurf/lib/python3.10/site-packages/osmnx/utils_geo.py:374: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for poly in multipoly:
/Users/khalid-alsheeb/opt/anaconda3/envs/kurf/lib/python3.10/site-packages/osmnx/utils_geo.py:374: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for poly in multipoly:


In [4]:
get_full_traffic_data(G)

In [ ]:
ox.plot_graph_folium(G, popup_attribute='current_speed_kmph')

In [ ]:
edges

In [5]:
ox.save_graphml(G)